**Write Neural Network from Scratch using Numpy**
<br/><br/>

# Define Gates (Add, Multiply)

**Some Notations**

$$
WX + b = \mathbf{S} \\
\frac{\partial{L}}{\partial{WX}} = d(WX) \\
\frac{\partial{L}}{\partial{S}} = d({S}) \\
$$

Math: http://cs231n.github.io/optimization-2/

In [1]:
%%writefile ./script/Propagation.py

import numpy as np
from util.im2col import *
from script.Optimization import *
class Mul:
    def forward(self, W, X):
        return np.dot(X, W)
    
    def backward(self, W, X, dWX):
        dW = np.dot( np.transpose(X), dWX )
        dX = np.dot( dWX, np.transpose (W))
        return dW, dX

Overwriting ./script/Propagation.py


In [2]:
%%writefile -a ./script/Propagation.py

class Add:
    def forward(self, WX, b):
        return WX + b

    def backward(self, WX, b, dS):
        dWX = dS * np.ones_like(WX, dtype=np.float64)
        db = np.dot(np.ones((1, dS.shape[0]), dtype=np.float64), dS)
        return db, dWX

Appending to ./script/Propagation.py


# Define Propagation

## Activations

$$
Activation(S) = Z \\
Input: 
\frac{\partial{L}}{\partial{Z}} = d(Z) \\
Output:
\frac{\partial{L}}{\partial{S}} = d(S) \\
$$


In [3]:
%%writefile -a ./script/Propagation.py

class ReLU:
    def forward(self, S):
        Z = S * (S > 0)
        return Z
    
    def backward(self, S, dZ):
        dS = 1. * (S > 0) * dZ
        return dS

Appending to ./script/Propagation.py


In [4]:
%%writefile -a ./script/Propagation.py

class Tanh:
    def forward(self, S):
        Z = np.tanh(S)
        return Z
    
    def backward(self, S, dZ):
        Z = self.forward(S)
        dS = (1.0 - np.square(Z)) * dZ
        return dS

Appending to ./script/Propagation.py


In [5]:
%%writefile -a ./script/Propagation.py

class Sigmoid:
    def forward(self, S):
        Z = 1. / (1.0 + np.exp(-S))
        return Z
    
    def backward(self, S, dZ):
        Z = self.forward(S)
        dS =(1 - Z) * Z * dZ
        return dS

Appending to ./script/Propagation.py


## Softmax

In [6]:
%%writefile -a ./script/Propagation.py

class Softmax:
    # For Training
    def __init__(self):
        self.num_examples = 0
    
    def forward(self, S):
        self.num_examples = S.shape[0]
        exp_S = np.exp(S)
        Z = exp_S / np.sum(exp_S, axis = 1, keepdims = True)
        return Z

    def backward(self, S, y): # Note: y instead of dZ
        probs = Z = self.forward(S)
        for i in range(len(y)):
            true_label = y[i]
            probs[i][true_label] -= 1 # see equation above
        dS = probs
        return dS
    
    # For evaluation    
    def forward_loss(self, Z, y):
        probs = Z
        log_probs = []
        for prob, true_label in zip(probs, y):
            log_probs.append(np.log(prob[true_label]))
        avg_cross_entropy_loss = - 1. / self.num_examples * np.sum(log_probs) # see equation above
        return avg_cross_entropy_loss
    
    # For prediction
    def predict(self, Z):
        return np.argmax(Z, axis = 1)

Appending to ./script/Propagation.py


$
for\ each\ sample\ i: 
$
$$
\hat{y_{k}} = softmax(S_1, S_2, ..., S_{k}),\ k\ is\ class\ index \\
\mathbf{E} = - \sum_{i=1}^N y_{ik} log(\hat{y_{ik}} )\\
\frac{\partial\mathbf{E}}{\partial S_{k}} = {\hat y}_{k} - y_{k},\ where\ y_{k} = 0, 1\\
$$

## Batch Normalization
- **Idea**: 
    - Normalize the inputs before activation function / after w*x + b
    - Differentiable operation
    - Robust to bad initialization
    
    
- **Advantages**: 
    - Faster training;
    - Allow scale and shift

<img src="https://kratzert.github.io/images/bn_backpass/bn_algorithm.PNG" width="400">

In [7]:
%%writefile -a ./script/Propagation.py

class BatchNorm:
    def __init__(self):
        self.cache = ()
        
    def forward(self, X, gamma, beta, eps):
        num_examples = X.shape[0]
        
        mu_B = 1. / num_examples * np.sum(X, axis = 0)
        X_mu = X - mu_B
        var_B = 1. / num_examples * np.sum(  X_mu ** 2, axis = 0 )
        sqrt_var_B = np.sqrt(var_B + eps)
        i_sqrt_var_B = 1. / sqrt_var_B
        X_hat =  X_mu * i_sqrt_var_B
        gammaX = gamma * X_hat
        DZ = gammaX + beta
        
        self.cache = (X_hat, X_mu, gamma, i_sqrt_var_B, sqrt_var_B, var_B, eps)
        return DZ
    
    def backward(self, dDZ):
        num_examples = dDZ.shape[0]
        X_hat, X_mu, gamma, i_sqrt_var_B, sqrt_var_B, var_B, eps = self.cache
        
        # scale and shift
        dbeta = np.sum(dDZ, axis = 0)
        dgammaX = dDZ
        dgamma = np.sum(dgammaX * X_hat, axis = 0)
        dXhat = dgammaX * gamma
        
        # Standardize
        di_sqrt_var_B = np.sum(dXhat * X_mu, axis = 0)
        d_x_mu_2 = dXhat * i_sqrt_var_B
        dsqrt_var_B = -1. / (sqrt_var_B ** 2) * di_sqrt_var_B
        dvar_B = 0.5 * 1. / np.sqrt(var_B + eps) * dsqrt_var_B

        # Batch variance
        dsquare = 1. / num_examples * np.ones_like(dDZ) * dvar_B
        d_x_mu_1 = 2 * X_mu * dsquare
        
        # Batch mean
        d_x_mu = d_x_mu_2 + d_x_mu_1 # d(f(x)g(x)) = f(x)g'(x) = f'(x)g(x)
        dmu = -1. * np.sum(d_x_mu, axis = 0)
        dx_2 = d_x_mu
        dx_1 = 1. / num_examples * np.ones_like(dDZ) * dmu
        dx = dx_2 + dx_1
        
        return dx, dgamma, dbeta

Appending to ./script/Propagation.py


## Convolutional layer

In [8]:
%%writefile -a ./script/Propagation.py

# Reference: https://github.com/wiseodd/hipsternet/blob/master/hipsternet
class Convolution:
    def __init__(self):
        self.cache = ()
        
    def forward(self, X, W, b, stride=1, padding=1):
        
        # W: (num_filters, num_channels, filter_h, filter_w)
        # X: (num_examples, num_channels, height, width)
        n_filters, d_filter, h_filter, w_filter = W.shape
        n_x, d_x, h_x, w_x = X.shape
        
        # Calculate Output Shape
        h_out = (h_x - h_filter + 2 * padding) / stride + 1
        w_out = (w_x - w_filter + 2 * padding) / stride + 1
        
        if not h_out.is_integer() or not w_out.is_integer():
            raise Exception('Invalid output dimension!')

        h_out, w_out = int(h_out), int(w_out)
        
        # Column-ize W and X
        # W_col: ( num_filters, num_channels * filter_h * filter_w) 
        # X_col: ( num_channels * filter_h * filter_w, h_out * w_out * num_examples)
        
        W_col = W.reshape(n_filters, -1)
        X_col = im2col_indices(X, h_filter, w_filter, padding=padding, stride=stride)
        
        # Matrix Multiply
        # W_col * X_col: (num_filters, h_out * w_out * num_examples)
        # b: (num_filters, 1)
        # WX + b: (num_filters, h_out * w_out * num_examples)
        
        out = np.matmul(W_col, X_col) + b
        out = out.reshape(n_filters, h_out, w_out, n_x)
        out = out.transpose(3, 0, 1, 2)

        # out: (num_examples, num_filters, h_out, w_out)
        self.cache = (X, W, b, stride, padding, X_col)

        return out


    def backward(self, dout):
        X, W, b, stride, padding, X_col = self.cache
        n_filter, d_filter, h_filter, w_filter = W.shape

        db = np.sum(dout, axis=(0, 2, 3))
        db = db.reshape(n_filter, -1)

        dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(n_filter, -1)
        dW = np.matmul(dout_reshaped,X_col.T)
        dW = dW.reshape(W.shape)

        W_reshape = W.reshape(n_filter, -1)
        dX_col = np.matmul(W_reshape.T,dout_reshaped)
        dX = col2im_indices(dX_col, X.shape, h_filter, w_filter, padding=padding, stride=stride)

        return dX, dW, db

Appending to ./script/Propagation.py


## Pooling Layer

In [9]:
%%writefile -a ./script/Propagation.py

class Maxpool:
    def forward(self, X_col):
        # X_reshaped: (num_examples * num_filters, 1, new_h_out, new_w_out)
        # for example: X.shape = (10,5,4,4), size = stride = 2, (new) h_out = w_out = 2
        # for example: X_reshaped = (50, 1, 2, 2)
        # X_col: (1 * size * size, num_filters(new_num_channels) * h_out * w_out * num_examples)
        # for example, X_col = (1*2*2, 5*2*2*10) = (4, 200)
        # max_idx.shape = out.shape = (200, )
        max_idx = np.argmax(X_col, axis=0) # for every (size * size) cells, select one max [0,1,2,3,1,2,...]
        out = X_col[max_idx, range(X_col.shape[1])]
        return out, max_idx

    def backward(self, dX_col, dout_col, max_idx):
        # Only max value got local gradient = 1
        dX_col[max_idx, range(dout_col.size)] = 1.0 * dout_col
        return dX_col

Appending to ./script/Propagation.py


In [10]:
%%writefile -a ./script/Propagation.py

class Pooling:
    def __init__(self, pool_fun = Maxpool()):
        self.cache = ()
        self.max_idx = ()
        self.pool_fun = pool_fun
        
    def forward(self, X, size = 2, stride = 2):
        # Calculate new shape after pooling
        
        # X_shape = (num_examples, num_filters, h_out, w_out) from CNN layer
        # for example: X = (10,5,4,4), size=stride=2, h_out=w_out=2
        n, d, h, w = X.shape
        h_out = (h - size) / stride + 1
        w_out = (w - size) / stride + 1

        if not w_out.is_integer() or not h_out.is_integer():
            raise Exception('Invalid output dimension!')

        h_out, w_out = int(h_out), int(w_out)

        # X_reshaped: (num_examples * num_filters, new_h_out, new_w_out)
        X_reshaped = X.reshape(n * d, 1, h, w)
        
        # X_col: (num_filters(new_num_channels) * size * size, h_out, w_out * num_examples)
        X_col = im2col_indices(X_reshaped, size, size, padding=0, stride=stride)

        out, self.max_idx = self.pool_fun.forward(X_col)
 
        out = out.reshape(h_out, w_out, n, d)
        out = out.transpose(2, 3, 0, 1)
        # out: (num_examples, num_filters (new_num_channels), new_h_out, new_w_out)
        
        self.cache = (X, size, stride, X_col)

        return out  
    
    
    def backward(self, dout):
        X, size, stride, X_col = self.cache
        n, d, w, h = X.shape

        dX_col = np.zeros_like(X_col)
        dout_col = dout.transpose(2, 3, 0, 1).ravel()

        dX = self.pool_fun.backward(dX_col, dout_col, self.max_idx)
        dX = col2im_indices(dX_col, (n * d, 1, h, w), size, size, padding=0, stride=stride)
        dX = dX.reshape(X.shape)

        return dX

Appending to ./script/Propagation.py


# Define Optimizer

In [11]:
%%writefile ./script/Optimization.py
import numpy as np
# Treat all elements of dX as a whole
#  Intuition: 
#  If gradient direction not changed, increase update, faster convergence
#  If gradient direction changed, reduce update, reduce oscillation

def VanillaUpdate(x, dx, learning_rate):
    x += -learning_rate * dx
    return x
        
# Vanilla version
#self.W += -learning_rate * self.dW + (- lambda_ * self.W)
#self.b += -learning_rate * self.db
#self.gamma += -learning_rate * self.dgamma
#self.beta  += -learning_rate * self.dbeta

def MomentumUpdate(x, dx, v, learning_rate, mu):
    v = mu * v - learning_rate * dx # integrate velocity, mu's typical value is about 0.9
    x += v # integrate position     
    return x, v

Overwriting ./script/Optimization.py


In [12]:
%%writefile -a ./script/Optimization.py

# Treat each element of dX adaptively
# Intuition:
# 1. Those dx receiving infrequent updates should have higher learning rate. vice versa 
# 2. We don't want: the gradients accumulate, and the learning rate monotically decrease, 
# 2. We want: modulates the learning rate of each weight based on the magnitudes of its gradient
# 3. Still want to use "momentum-like" update to get a smooth gradient

# 1. AdaGrad
def AdaGrad(x, dx, learning_rate, cache, eps):
    cache += dx**2
    x += - learning_rate * dx / (np.sqrt(cache) + eps) # (usually set somewhere in range from 1e-4 to 1e-8)
    return x, cache
    
# 1+2. RMSprop
def RMSprop(x, dx, learning_rate, cache, eps, decay_rate): #Here, decay_rate typical values are [0.9, 0.99, 0.999]
    cache = decay_rate * cache + (1 - decay_rate) * dx**2
    x += - learning_rate * dx / (np.sqrt(cache) + eps)
    return x, cache
    
# 1+2+3. Adam
def Adam(x, dx, learning_rate, m, v, t, beta1, beta2, eps):
    m = beta1*m + (1-beta1)*dx # Smooth gradient
    #mt = m / (1-beta1**t) # bias-correction step
    v = beta2*v + (1-beta2)*(dx**2) # keep track of past updates
    #vt = v / (1-beta2**t) # bias-correction step
    x += - learning_rate * m / (np.sqrt(v) + eps) # eps = 1e-8, beta1 = 0.9, beta2 = 0.999   
    return x, m, v

Appending to ./script/Optimization.py


In [13]:
%%writefile -a ./script/Optimization.py

class WeightUpdate:
    def __init__(self, init_value):
        self.val = init_value
        self.cache = np.zeros_like(self.val, dtype=np.float64)
        self.m = np.zeros_like(self.val, dtype=np.float64)
        self.v = np.zeros_like(self.val, dtype=np.float64)
        self.t = 0
    
    def Update(self, d, learning_rate, lambda_ , method):
        
        old_val = self.val
        if method == 'Vanilla':
            self.val = VanillaUpdate(self.val, d, learning_rate)
        elif method == 'MomentumUpdate':
            self.val, self.v = MomentumUpdate(self.val, d, self.v, learning_rate, mu = 0.9)
        elif method == 'AdaGrad':
            self.val, self.cache = AdaGrad(self.val, d, learning_rate, self.cache, eps = 1e-5)
        elif method == 'RMSprop':
            self.val, self.cache = AdaGrad(self.val, d, learning_rate, self.cache, eps = 1e-5, decay_rate = 0.99)
        elif method == 'Adam':
            self.val, self.m, self.v = Adam(self.val, d, learning_rate, self.m, self.v, self.t, beta1 = 0.9, beta2 = 0.999, eps = 1e-8)  
            self.t += 1
            
        # Regularization
        self.val -= lambda_ * old_val
        return self.val

Appending to ./script/Optimization.py


# Define Layer

## Base Layer

In [14]:
%%writefile ./script/Layer.py
from script.Propagation import *
class BaseLayer:
    def __init__(self):
        self.isfirst = False
        self.islast = False
        self.before = None
        self.after = None
        self.gamma, self.beta = 1,0
        self.dgamma, self.dbeta = 0,0

    def set_first_layer(self, input):
        self.isfirst = True
        self.X = input
        
    def set_last_layer(self, y):
        self.islast = True
        self.y = y
    
    def initialize_Wb(self):
        raise NotImplementedError()

    def forward_propagation(self):
        raise NotImplementedError()
            
    def backward_propagation(self):
        raise NotImplementedError()
    
    def update_weight(self, learning_rate, lambda_ , method):
        
        # Create variable list
        self.weights = [self.W, self.b, self.gamma, self.beta]
        self.ds = [self.dW, self.db, self.dgamma, self.dbeta]

        # First Time
        if not hasattr(self, 'updates'):      
            self.updates = []
            for weight in self.weights:
                self.updates.append(WeightUpdate(weight))
        
        # Calculate update for each iteration
        new_weights = []
        for weight_update, d in zip(self.updates, self.ds):
            new_weights.append(weight_update.Update(d, learning_rate, lambda_, method))
        
        # Update weights
        self.W, self.b, self.gamma, self.beta = new_weights

Overwriting ./script/Layer.py


## Fully-Connected Layer

In [15]:
%%writefile -a ./script/Layer.py

class FC(BaseLayer):
    def __init__(self, activation_function, num_neurons, batch_norm = False, dropout_p = 1.0): # p = 1 means no dropout
        super().__init__()
        self.dim = num_neurons
        self.activation = activation_function
        self.batch_norm = batch_norm
        if batch_norm:
            self.batchnorm = BatchNorm()
        self.p = dropout_p
        self.name = 'FC'
        
    def initialize_Wb(self):
        before_dim = self.X.shape[1]
        self.W = np.random.randn(before_dim, self.dim) / np.sqrt(before_dim) # see notes above
        self.b = np.random.randn(self.dim).reshape(1, self.dim) # see notes above
        
    def forward_propagation(self):
        if not self.isfirst:
            self.X = self.before.Z
        
        if not hasattr(self, 'W'):
            self.initialize_Wb() 
            
        self.mask = np.random.rand(*self.W.shape) < self.p / self.p
        self.W *= self.mask
        self.WX = Mul().forward( self.W, self.X )
        self.S = Add().forward( self.WX, self.b)
        if self.batch_norm:
            self.SZ = self.batchnorm.forward( self.S, self.gamma, self.beta, eps = 0)
        else:
            self.SZ = self.S
        self.Z = self.activation.forward(self.SZ)
            
    def backward_propagation(self):
        if self.islast:
            self.dZ = self.y
        
        self.dSZ = self.activation.backward(self.SZ, self.dZ)
        if self.batch_norm:
            self.dS, self.dgamma, self.dbeta = self.batchnorm.backward(self.dSZ)
        else:
            self.dS = self.dSZ
            self.dgamma, self.dbeta = 0,0
        self.db, self.dWX = Add().backward(self.WX, self.b, self.dS)
        self.dW, self.dX = Mul().backward(self.W, self.X, self.dWX)
        self.dW *= self.mask
        
        if not self.isfirst:
            self.before.dZ = self.dX
            


Appending to ./script/Layer.py


## Softmax Layer

In [16]:
%%writefile -a ./script/Layer.py

class Softmax_Classifier(FC):
    def __init__(self, activation_function, num_neurons, batch_norm = False, dropout_p = 1.0):
        super().__init__(activation_function, num_neurons, batch_norm, dropout_p)
        self.name = 'Softmax'
        
    def calculate_loss(self):
        loss = self.activation.forward_loss(self.Z, self.y)
        return loss
            
    def predict(self):
        return self.activation.predict(self.Z)
    
    def calculate_acc(self): 
        pred = self.predict()
        return sum( pred == self.y ) / len(self.y)  


Appending to ./script/Layer.py


## CNN Layer

In [17]:
%%writefile -a ./script/Layer.py
  

class CNN(BaseLayer):
    def __init__(self, activation_function, params):
        super().__init__()
        self.activation = activation_function
        self.conv = Convolution()
        self.pooling = Pooling()
        self.name = 'CNN'
       
        self.num_filters,  self.filter_h, self.filter_w = params['num_filters'], params['filter_h'],params['filter_w']
        self.filter_stride, self.filter_padding = params['filter_stride'],params['filter_padding']
        self.pooling_size, self.pooling_stride = params['pooling_size'],params['pooling_stride']
    
    def initialize_Wb(self):
        # W: (num_filters, num_channels, filter_h, filter_w)
        # X: (num_examples, num_channels, height, width)
        # b: (num_filters, 1)
        if self.isfirst:
            self.num_channels = self.X.shape[-3] 
        else:
            self.num_channels = self.before.num_filters
        self.W = np.random.randn(self.num_filters, self.num_channels, self.filter_h, self.filter_w) / np.sqrt(self.num_filters / 2.)
        self.b = np.random.randn(self.num_filters).reshape(self.num_filters,1)
        
    def forward_propagation(self):
        if not self.isfirst:
            self.X = self.before.Z

        if not hasattr(self, 'W'):
            self.initialize_Wb() 
            
        self.D = self.conv.forward(self.X, self.W, self.b, 
                                   stride = self.filter_stride, 
                                   padding = self.filter_padding)
        self.Z_conv = self.activation.forward(self.D)
        self.Z_conv_pool = self.pooling.forward(self.Z_conv, 
                                                size = self.pooling_size, 
                                                stride = self.pooling_stride)
        
        if self.after.name != 'CNN':
            self.Z_pool_flat = self.Z_conv_pool.ravel().reshape(self.X.shape[0], -1)
            self.Z = self.Z_pool_flat
        else:
            self.Z = self.Z_conv_pool
    
    def backward_propagation(self):
        if self.after.name != 'CNN':
            self.dZ_conv_pool = self.dZ.ravel().reshape(self.Z_conv_pool.shape)
        else:
            self.dZ_conv_pool = self.dZ
            
        self.dZ_conv = self.pooling.backward(self.dZ_conv_pool)
        self.dX, self.dW, self.db  = self.conv.backward(self.dZ_conv)
        
        if not self.isfirst:
            self.before.dZ = self.dX

Appending to ./script/Layer.py


# Define Network

In [22]:
%%writefile ./script/Network.py
import numpy as np
from script.Layer import *
class Network:
    def __init__(self):
        self.layers = []
        self.input = []
        self.y = []
        
    def add(self, new_layer):
        if self.layers:
            self.layers[-1].after = new_layer
            new_layer.before = self.layers[-1]
        self.layers.append(new_layer)
    
    def load_data(self, input, y):
        self.layers[0].set_first_layer(input)
        self.layers[-1].set_last_layer(y)
        
    def initialize(self, input, y, batch_size):
        self.input = input
        self.y = y
        self.load_data(input[:batch_size,:], y[:batch_size])
        #for layer in self.layers:
        #    layer.initialize_Wb()

    def train(self, num_iter, learning_rate, batch_size, rand_, lambda_, optimizer = 'Vanilla', Val_X = None, Val_y = None, 
             CAL_STEP = 100, PRINT_STEP = 100):
        
        train_loss = []
        train_acc = []
        val_loss = []
        val_acc = []
        
        for i in range(num_iter):    
            # Calculate batch index
            if not rand_:
                idx = list(range(self.input.shape[0]))
            else:
                idx = np.random.randint(self.input.shape[0], size = batch_size)
            
            self.load_data(self.input[idx,:], self.y[idx])
            
            # Forward Propagation
            for layer in self.layers:
                layer.forward_propagation()
                
            # Print Traing Acc/Loss
            if (i % CAL_STEP == 0):
                t_loss = self.layers[-1].calculate_loss()
                t_acc  = self.layers[-1].calculate_acc()
                train_loss.append(t_loss)
                train_acc.append(t_acc)
                
            if (i % PRINT_STEP == 0):
                print('Train at Iter {0:2d}: loss - {1:.3f}, Acc - {2:.3f}'.format(i, t_loss, t_acc))
                
            # Backward Propagation
            for layer in self.layers[::-1]:
                layer.backward_propagation()
                layer.update_weight(learning_rate, lambda_ = lambda_ , method = optimizer)
            
            # Print Validation Acc/Loss
            if (i % CAL_STEP == 0 and Val_X is not None):
                v_acc, v_loss = self.evaluate(Val_X, Val_y)
                val_loss.append(v_loss)
                val_acc.append(v_acc) 
                
            if (i % PRINT_STEP == 0 and Val_X is not None):
                print('Validation at Iter {0:2d}: loss - {1:.3f}, Acc - {2:.3f}'.format(i, v_loss, v_acc))

        # Finally return loss list
        return train_loss, train_acc, val_loss, val_acc
    
    def predict(self, X):
        self.load_data(X, y = None)
        for layer in self.layers:
            layer.forward_propagation()
        return layers[-1].predict()
        
    def evaluate(self, X, y):
        self.load_data(X, y)
        for layer in self.layers:
            layer.forward_propagation()
        loss = self.layers[-1].calculate_loss()
        acc  = self.layers[-1].calculate_acc()
        return acc, loss

Overwriting ./script/Network.py


# Test

In [21]:
from script.Propagation import *
from script.Layer import *

# Test Case
# Default padding: SAME
NUM_EXAMPLES = 10
NUM_CHANNELS = 7
NUM_FILTERS = 5
INPUT_HEIGHT = 4
INPUT_WIDTH =  4
FILTER_HEIGHT = 1
FILTER_WIDTH = 1
STRIDE = 1
PAD = 0

n_x, d_x, h_x, w_x  = x_shape = (NUM_EXAMPLES, NUM_CHANNELS, INPUT_HEIGHT, INPUT_WIDTH) # num_examples, depth(channels), height, width
n_filters, d_filter, h_filter, w_filter = w_shape = (NUM_FILTERS, NUM_CHANNELS, FILTER_HEIGHT, FILTER_WIDTH) # num_filters, depth(channels), kernel_height, kernel_width
b_shape = (NUM_FILTERS, 1)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=np.prod(b_shape)).reshape(b_shape)

# Test 1:

params = {'num_filters': NUM_FILTERS,
          'filter_h': FILTER_HEIGHT,
          'filter_w': FILTER_WIDTH,
          'filter_stride': STRIDE,
          'filter_padding': PAD,
          'pooling_size': 2,
          'pooling_stride': 2
         }

cnn = CNN(activation_function= ReLU(), params = params )

# Test 2:
conv = Convolution()
relu = ReLU()
pool = Pooling()
D = conv.forward(x, w, b, stride = STRIDE, padding = PAD)
Z_conv = relu.forward(D)
Z_conv_pool = pool.forward(Z_conv, size = 2, stride = 2)
Z_pool_flat = Z_conv_pool.ravel().reshape(x.shape[0], -1)
print(x.shape, D.shape, Z_conv.shape, Z_conv_pool.shape, Z_pool_flat.shape)

(10, 7, 4, 4) (10, 5, 4, 4) (10, 5, 4, 4) (10, 5, 2, 2) (10, 20)
